<a href="https://colab.research.google.com/github/Jgallear/CSSP_brazil_23_24/blob/main/seasonal_null_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# script to create a seasonal null model to compare with each of the ML models
# seasonal null model just predicts VHI as the mean value of previous months VHI

In [2]:
!pip install cartopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 49.7 MB/s eta 0:00:00


In [3]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
from sklearn.inspection import PartialDependenceDisplay
from shapely.geometry import Point
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
#from rasterio import CRS
from shapely.geometry import mapping
import xarray as xr
from statsmodels.graphics.tsaplots import plot_acf
from keras import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import functools
import pickle
import cartopy.feature as cf
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from sklearn.metrics import r2_score

In [4]:
df = pd.read_csv('/content/drive/MyDrive/VHI_spei_rzsm_dataset.csv')

In [5]:
print(df)

           lon     lat      vhi-1   precip-1      rzsm-1  spei1-1m  spei2-1m  \
0      -53.375 -32.125  70.414172  259.20056   98.021910  2.192168  1.313241   
1      -53.375 -32.125  79.323750  146.58607   98.162980  1.228278  2.453098   
2      -53.375 -32.125  77.111895  184.82845   97.594270  1.472914  1.588110   
3      -53.375 -32.125  64.150561  108.86273   97.605330  1.680747  1.751371   
4      -53.375 -32.125  54.270359  145.90760  100.000000  1.641170  1.900034   
...        ...     ...        ...        ...         ...       ...       ...   
577682 -47.625  -0.875  41.102345  170.38158   66.929490  0.324701  0.572972   
577683 -47.625  -0.875  49.922513  294.35620   80.138200  1.130223  0.881931   
577684 -47.625  -0.875  52.101102  125.48111   88.888840  2.664571  2.250521   
577685 -47.625  -0.875  46.753572   24.13338   77.834076  2.523025  3.002636   
577686 -47.625  -0.875  48.627606   71.58264   69.860950  1.286047  2.275886   

        spei3-1m   spi01-1   spi02-1   

In [6]:
# Need to get r2 score and MAE per year
def groupby_year(df):
  df_list=[]
  for index,df in df.groupby(['year']):
    df_list.append(df)
  return df_list

def groupby_coordinates(df):
  df_list=[]
  for index,df in df.groupby(['lat','lon']):
    df_list.append(df)
  return df_list

In [7]:
# Get average VHI for each month per location
#av_monthly_VHI = df.groupby(['month']).mean()['VHI'].values

### Want to group coordinates into list, then groupby month and average
VHI_per_coord_list = groupby_coordinates(df)

In [8]:
def coordinate_monthly_average(df_list):
  coord_m_av=[]
  lats_lists=[]
  lons_lists=[]
  df_2_list=[]
  for i in range(len(df_list)):
    dfi = df_list[i]
    ###
    av_monthly_VHI_i = dfi.groupby(['month']).mean()['VHI'].values
    lons_l = [dfi['lon'].values[0]]*len(av_monthly_VHI_i)
    lats_l = [dfi['lat'].values[0]]*len(av_monthly_VHI_i)
    lats_lists.append(lats_l)
    lons_lists.append(lons_l)
    month_no = np.arange(1,12+1)
    # could combine here in dataframe
    df_2 = pd.DataFrame({'lat':lats_l,'lon':lons_l,'month':month_no,'VHI prediction':av_monthly_VHI_i})
    #print(df_2)
    df_2_list.append(df_2)
    ###
    coord_m_av.append(av_monthly_VHI_i)
  return pd.concat(df_2_list)


In [9]:
coord_m_av_VHIs = coordinate_monthly_average(VHI_per_coord_list)

In [10]:
print(coord_m_av_VHIs)

       lat     lon  month  VHI prediction
0  -32.125 -53.375      1       56.846723
1  -32.125 -53.375      2       61.658084
2  -32.125 -53.375      3       57.289225
3  -32.125 -53.375      4       49.464822
4  -32.125 -53.375      5       47.680626
..     ...     ...    ...             ...
7   -0.875 -47.625      8       47.093687
8   -0.875 -47.625      9       43.050601
9   -0.875 -47.625     10       43.659320
10  -0.875 -47.625     11       46.048799
11  -0.875 -47.625     12       46.180196

[29964 rows x 4 columns]


In [11]:
print(df)

           lon     lat      vhi-1   precip-1      rzsm-1  spei1-1m  spei2-1m  \
0      -53.375 -32.125  70.414172  259.20056   98.021910  2.192168  1.313241   
1      -53.375 -32.125  79.323750  146.58607   98.162980  1.228278  2.453098   
2      -53.375 -32.125  77.111895  184.82845   97.594270  1.472914  1.588110   
3      -53.375 -32.125  64.150561  108.86273   97.605330  1.680747  1.751371   
4      -53.375 -32.125  54.270359  145.90760  100.000000  1.641170  1.900034   
...        ...     ...        ...        ...         ...       ...       ...   
577682 -47.625  -0.875  41.102345  170.38158   66.929490  0.324701  0.572972   
577683 -47.625  -0.875  49.922513  294.35620   80.138200  1.130223  0.881931   
577684 -47.625  -0.875  52.101102  125.48111   88.888840  2.664571  2.250521   
577685 -47.625  -0.875  46.753572   24.13338   77.834076  2.523025  3.002636   
577686 -47.625  -0.875  48.627606   71.58264   69.860950  1.286047  2.275886   

        spei3-1m   spi01-1   spi02-1   

In [12]:
VHI_dat = pd.DataFrame({'lon':df['lon'],'lat':df['lat'],'VHI':df['VHI'],'month':df['month'],'year':df['year']})

In [13]:
# Get fixed years
fixed_years = [2016, 2011, 2019, 2015, 2006]

In [14]:
VHI_dat_test_years = VHI_dat[VHI_dat['year'].isin(fixed_years)]

In [15]:
# test years groupby year then for each year do pandas merge on month and coordinate with coord_m_av_VHIs
def merge_avm_vhi(vhi_df,avm_vhi_df):
  vhi_df_yr = groupby_year(vhi_df)
  # for each year merge with average monthly vhi
  merged_dfs_list=[]
  for i in range(len(vhi_df_yr)):
    vdfi = vhi_df_yr[i]
    vdfi_merged = vdfi.merge(avm_vhi_df,on=['lat','lon','month'])
    merged_dfs_list.append(vdfi_merged)
  return pd.concat(merged_dfs_list)


In [16]:
VHI_pred_df = merge_avm_vhi(VHI_dat_test_years,coord_m_av_VHIs)

<ipython-input-6-9b2e1a08fd0b>:4: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for index,df in df.groupby(['year']):


In [17]:
print(VHI_pred_df)

          lon     lat        VHI  month  year  VHI prediction
0     -53.375 -32.125  53.849729      1  2006       56.846723
1     -53.375 -32.125  62.237937      2  2006       61.658084
2     -53.375 -32.125  60.625543      3  2006       57.289225
3     -53.375 -32.125  64.313523      4  2006       49.464822
4     -53.375 -32.125  62.954366      5  2006       47.680626
...       ...     ...        ...    ...   ...             ...
30531 -47.625  -0.875  40.887218      7  2019       45.074835
30532 -47.625  -0.875  50.892596      8  2019       47.093687
30533 -47.625  -0.875  43.292766      9  2019       43.050601
30534 -47.625  -0.875  46.295286     10  2019       43.659320
30535 -47.625  -0.875  52.592186     11  2019       46.048799

[152680 rows x 6 columns]


In [18]:
def yearly_metrics(df):
  df_list = groupby_year(df)
  MAE_values=[]
  r2_values=[]
  year=[]
  for i in range(len(df_list)):
    MAE_values.append(mean_absolute_error(df_list[i]['VHI'],df_list[i]['VHI prediction']))
    r2_values.append(r2_score(df_list[i]['VHI'],df_list[i]['VHI prediction']))
    year.append(df_list[i]['year'].values[0])
  return pd.DataFrame({'MAE':MAE_values,'R2':r2_values})

In [19]:
av_seasonal_model_df = yearly_metrics(VHI_pred_df)

<ipython-input-6-9b2e1a08fd0b>:4: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for index,df in df.groupby(['year']):


In [20]:
print(av_seasonal_model_df)

         MAE        R2
0   9.747020 -0.147920
1   6.997027  0.262999
2   8.304895  0.423711
3  10.337524  0.300688
4   6.589992  0.469473


In [21]:
# save df
av_seasonal_model_df.to_csv('/content/drive/MyDrive/model_outs/seasonal_average_model_5y.csv',index=False)